In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import math
import folium
from geopy.geocoders import Nominatim
import json
import os
from folium.plugins import Search

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Organisation spreadsheet, invoicing of credit notes and reimbursements

In [ ]:
df = pd.read_excel("data/pandas_organizace.xls")

df = df.rename(columns={'Datum poslední vystavené faktury': 'Datum poslední faktury'})
df = df.rename(columns={'Výše poslední vystavené fa': 'Výše poslední faktury'})
df = df.rename(columns={'Datum poslední úhrady IČ': 'Datum poslední úhrady'})

df["Adresa 1"] = df["Ulice s čísly"] + ", " + df["Místo sídla"] + ", " + df["PSČ"] + ", " + df["ISO kód"]
df["Adresa 2"] = df["Místo sídla"] + ", " + df["PSČ"] + ", " + df["ISO kód"]

df_f = pd.read_excel("data/pandas_faktury.xls")

df_f = df_f.rename(columns={'Datum případu (DMR)': 'Datum faktury'})
df_f = df_f.rename(columns={'Valuty celkem po zaokrouhlení': 'Faktura s DPH'})
df_f = df_f.rename(columns={'Poznámka 3 - Vše': 'Sleva'})

df_u = pd.read_excel("data/pandas_úhrady.xls")

df_u = df_u.rename(columns={'Datum (DMR)': 'Datum úhrady'})
df_u = df_u.rename(columns={'Odpovědná osoba.1': 'Odpovědná osoba'})
df_u = df_u.rename(columns={'Částka úhrady druhová': 'Částka úhrady'})

Add columns from DF-F - Total balance, Date of last invoice, Amount of last invoice, Last invoice series and add columns from DF_U - Date of last payment, Amount of last payment = 6 columns + KEY SLOUPEC

In [ ]:
df['Key'] = np.where(df['IČO'].notna() & (df['IČO'] != ''), df['IČO'], df['DIČ DPH'])
df_f['Key'] = np.where(df_f['IČO'].notna() & (df_f['IČO'] != ''), df_f['IČO'], df_f['DIČ DPH'])
df_u['Key'] = np.where(df_u['IČO'].notna() & (df_u['IČO'] != ''), df_u['IČO'], df_u['DIČ DPH'])

saldo_df = df_f.groupby('Key', as_index=False)['Saldo'].sum().rename(columns={'Saldo': 'Celkové saldo'})

def convert_sleva_to_float(x):
   
    if pd.isnull(x):
        return np.nan
    
    s = str(x).lower()
    s = re.sub(r'[^0-9.,]', '', s)
    s = s.replace(',', '.')
    
    try:
        val = float(s)
    except ValueError:
        val = np.nan
    return val

df_f['SlevaNum'] = df_f['Sleva'].apply(convert_sleva_to_float)

nejcastejsi_sleva_series = (
    df_f
    .dropna(subset=['SlevaNum'])
    .groupby('Key')['SlevaNum']
    .agg(lambda x: x.value_counts().idxmax())
)

nejcastejsi_sleva_df = nejcastejsi_sleva_series.reset_index().rename(columns={'SlevaNum': 'Nejčastější sleva'})


today = pd.to_datetime('today').normalize()
df_f['DaysOverdue'] = (today - df_f['Splatnost']).dt.days

has_uhrada_set = set(zip(df_u['Key'], df_u['Řada'], df_u['Poř.č.']))

promlceno_mask = (
    (df_f['DaysOverdue'] > 1095) &
    (~df_f.apply(lambda row: (row['Key'], row['Řada'], row['Poř.č.']) in has_uhrada_set, axis=1))
)
promlceno_df = (
    df_f.loc[promlceno_mask]
    .groupby('Key', as_index=False)['Saldo']
    .sum()
    .rename(columns={'Saldo': 'Promlčeno'})
)

pred_promlcenim_mask = (
    (df_f['DaysOverdue'].between(1000, 1095)) &
    (~df_f.apply(lambda row: (row['Key'], row['Řada'], row['Poř.č.']) in has_uhrada_set, axis=1))
)
pred_promlcenim_df = (
    df_f.loc[pred_promlcenim_mask]
    .groupby('Key', as_index=False)['Saldo']
    .sum()
    .rename(columns={'Saldo': 'Před promlčením'})
)

nad_90_df = (
    df_f.loc[df_f['DaysOverdue'] > 90]
    .groupby('Key', as_index=False)['Saldo']
    .sum()
    .rename(columns={'Saldo': 'Nad 90 dní po splatnosti'})
)

do_90_df = (
    df_f.loc[df_f['DaysOverdue'].between(61, 90)]
    .groupby('Key', as_index=False)['Saldo']
    .sum()
    .rename(columns={'Saldo': 'Do 90 dní po splatnosti'})
)

do_60_df = (
    df_f.loc[df_f['DaysOverdue'].between(31, 60)]
    .groupby('Key', as_index=False)['Saldo']
    .sum()
    .rename(columns={'Saldo': 'Do 60 dní po splatnosti'})
)

do_30_df = (
    df_f.loc[df_f['DaysOverdue'].between(1, 30)]
    .groupby('Key', as_index=False)['Saldo']
    .sum()
    .rename(columns={'Saldo': 'Do 30 dní po splatnosti'})
)

ve_splatnosti_df = (
    df_f.loc[df_f['DaysOverdue'] <= 0]
    .groupby('Key', as_index=False)['Saldo']
    .sum()
    .rename(columns={'Saldo': 'Ve splatnosti'})
)

agg_df_f = (
    saldo_df
    .merge(nejcastejsi_sleva_df, on='Key', how='left')
    .merge(promlceno_df,         on='Key', how='left')
    .merge(pred_promlcenim_df,   on='Key', how='left')
    .merge(nad_90_df,            on='Key', how='left')
    .merge(do_90_df,             on='Key', how='left')
    .merge(do_60_df,             on='Key', how='left')
    .merge(do_30_df,             on='Key', how='left')
    .merge(ve_splatnosti_df,     on='Key', how='left')
)

idx_f = df_f.groupby('Key')['Datum faktury'].idxmax()
latest_invoices = df_f.loc[idx_f, ['Key', 'Datum faktury', 'Faktura s DPH', 'Řada']].rename(
    columns={
        'Datum faktury': 'Datum poslední faktury',
        'Faktura s DPH': 'Výše poslední faktury',
        'Řada': 'Poslední fakturační řada'
    }
)

agg_df_f = agg_df_f.merge(latest_invoices, on='Key', how='left')

idx_u = df_u.groupby('Key')['Datum úhrady'].idxmax()
agg_df_u = df_u.loc[idx_u, ['Key', 'Datum úhrady', 'Částka úhrady']].rename(
    columns={
        'Datum úhrady': 'Datum poslední úhrady',
        'Částka úhrady': 'Výše poslední úhrady'
    }
)

df = df.merge(agg_df_f, on='Key', how='left')
df = df.merge(agg_df_u, on='Key', how='left')

In DF from DF_F create columns for invoices for the last floating 4 years - Turnover XXXX, Number of XXXX pcs, Average XXX and in DF from DF_U create columns for payments for the last floating 4 years - Payments XXXX, Cash-flow XXXX in %

In [ ]:
current_year = datetime.now().year
years = range(current_year - 3, current_year + 1)

df_f['Rok'] = df_f['Datum faktury'].dt.year

filtered_df_f = df_f[df_f['Rok'].isin(years)]

agg_f = (
    filtered_df_f
    .groupby(['Key', 'Rok'], as_index=False)
    .agg(
        Obrat=('Faktura s DPH', 'sum'),
        Počet=('Faktura s DPH', 'count'),
        Průměr=('Faktura s DPH', 'mean')
    )
)

pivoted_f = agg_f.pivot(index='Key', columns='Rok', values=['Obrat', 'Počet', 'Průměr'])

pivoted_f.columns = [f"{metric}_{year}" for metric, year in pivoted_f.columns]


df_u['Rok'] = df_u['Datum úhrady'].dt.year
filtered_df_u = df_u[df_u['Rok'].isin(years)]


agg_u = (
    filtered_df_u
    .groupby(['Key', 'Rok'], as_index=False)
    .agg(
        Úhrady=('Částka úhrady', 'sum')
    )
)

pivoted_u = agg_u.pivot(index='Key', columns='Rok', values=['Úhrady'])

pivoted_u.columns = [f"{metric}_{year}" for metric, year in pivoted_u.columns]

combined = pivoted_f.merge(pivoted_u, on='Key', how='left')

for y in years:
    obrat_col = f"Obrat_{y}"
    uhrady_col = f"Úhrady_{y}"
    cf_col = f"C-F_{y}"
    if obrat_col in combined.columns and uhrady_col in combined.columns:
        combined[cf_col] = (combined[uhrady_col] / combined[obrat_col] * 100).round()
    else:
        combined[cf_col] = np.nan

metrics_order = ['Obrat', 'Počet', 'Průměr', 'Úhrady', 'C-F']
ordered_cols = []
for y in years:
    for m in metrics_order:
        col_name = f"{m}_{y}"
        if col_name in combined.columns:
            ordered_cols.append(col_name)

combined = combined.reindex(columns=ordered_cols)

df = df.merge(combined, on='Key', how='left')

v DF úprava formátů a přidání měny

In [ ]:
def format_currency(value, iso):
   
    if pd.isnull(value) or value == 0:
        return ''
    9
    s = f"{value:,.2f}" 
    s = s.replace(',', 'X').replace('.', ',').replace('X', ' ')

    s += " Kč" if iso == "CZ" else " Eur"
    return s

def format_count(value):
    if pd.isnull(value):
        return ''
    if value == 0:
        return ''
    return f"{int(value):,d} ks".replace(',', ' ')

def format_percentage(value):
   
    if pd.isnull(value) or math.isinf(value):
        return ''
    if value == 0:
        return ''

    return f"{int(round(value))} %"

def format_date(value):
    if pd.isnull(value):
        return ''
    return value.strftime("%d.%m.%Y")

def format_row(row):
    iso = row['ISO kód'] if 'ISO kód' in row else 'CZ'

   
    if 'Celkové saldo' in row:
        row['Celkové saldo'] = format_currency(row['Celkové saldo'], iso)
    if 'Výše poslední faktury' in row:
        row['Výše poslední faktury'] = format_currency(row['Výše poslední faktury'], iso)
    if 'Výše poslední úhrady' in row:
        row['Výše poslední úhrady'] = format_currency(row['Výše poslední úhrady'], iso)
    if 'Datum poslední faktury' in row:
        row['Datum poslední faktury'] = format_date(row['Datum poslední faktury'])
    if 'Datum poslední úhrady' in row:
        row['Datum poslední úhrady'] = format_date(row['Datum poslední úhrady'])

   
    if 'Poslední fakturační řada' in row:
        if pd.notnull(row['Poslední fakturační řada']):
            row['Poslední fakturační řada'] = str(int(row['Poslední fakturační řada']))
        else:
            row['Poslední fakturační řada'] = ''

       if 'Promlčeno' in row:
        row['Promlčeno'] = format_currency(row['Promlčeno'], iso)
   
    if 'Před promlčením' in row:
        row['Před promlčením'] = format_currency(row['Před promlčením'], iso)
    
    if 'Nad 90 dní po splatnosti' in row:
        row['Nad 90 dní po splatnosti'] = format_currency(row['Nad 90 dní po splatnosti'], iso)
    
    if 'Do 90 dní po splatnosti' in row:
        row['Do 90 dní po splatnosti'] = format_currency(row['Do 90 dní po splatnosti'], iso)
    
    if 'Do 60 dní po splatnosti' in row:
        row['Do 60 dní po splatnosti'] = format_currency(row['Do 60 dní po splatnosti'], iso)
    
    if 'Do 30 dní po splatnosti' in row:
        row['Do 30 dní po splatnosti'] = format_currency(row['Do 30 dní po splatnosti'], iso)
    
    if 'Ve splatnosti' in row:
        row['Ve splatnosti'] = format_currency(row['Ve splatnosti'], iso)

   
    if 'Nejčastější sleva' in row:
        row['Nejčastější sleva'] = format_percentage(row['Nejčastější sleva'])

    
    for col in row.index:
        
        if col.startswith('Obrat_') or col.startswith('Průměr_') or col.startswith('Úhrady_'):
            row[col] = format_currency(row[col], iso)

        
        elif col.startswith('Počet_'):
            row[col] = format_count(row[col])

        
        elif col.startswith('C-F_'):
            row[col] = format_percentage(row[col])

    return row


df = df.apply(format_row, axis=1)

INTERACTIVE MAP

In [ ]:
current_date = datetime.now().strftime("%d.%m.%Y")

current_year = datetime.now().year
years = range(current_year - 3, current_year + 1)

def to_serializable(val):
    if pd.isnull(val):
        return "N/A"
    if isinstance(val, (np.integer, np.int64)):
        return int(val)
    if isinstance(val, (np.floating, np.float64)):
        return float(val)
    if isinstance(val, pd.Timestamp):
        return val.strftime("%Y-%m-%d")
    return str(val)

json_file = "data/geocoded_addresses.json"

def load_json(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    return {}

def save_json(data, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

geocoded_data = load_json(json_file)
geolocator = Nominatim(user_agent="map_app")

def get_coordinates(address):
    if address in geocoded_data:
        return geocoded_data[address]
    try:
        location = geolocator.geocode(address)
        if location:
            geocoded_data[address] = (location.latitude, location.longitude)
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception:
        return None, None

def get_coordinates_with_fallback(row):
    lat, lon = get_coordinates(row["Adresa 1"])
    if lat is not None and lon is not None:
        return lat, lon, "Adresa 1"
    else:
        fallback_address = row.get("Adresa 2", None)
        if pd.notna(fallback_address):
            lat_f, lon_f = get_coordinates(fallback_address)
            if lat_f is not None and lon_f is not None:
                return lat_f, lon_f, "Adresa 2"
    return None, None, None

df["Latitude"], df["Longitude"], df["GeocodedFrom"] = zip(*df.apply(get_coordinates_with_fallback, axis=1))
save_json(geocoded_data, json_file)

df["Firma_Adresa"] = df["Firma"].fillna('') + " " + df["Adresa 1"].fillna('')

map = folium.Map(location=[49.8175, 15.4730], zoom_start=7, zoom_control=False)

map.get_root().html.add_child(folium.Element("""
<style>
.leaflet-control-attribution {
    display: none !important;
}
</style>
"""))

coordinate_counts = {}
nezmapovane_radky = []
pocet_pridanych_znacek = 0

person_color_map = {
    np.nan: "black",
    "Lelek Michal": "yellow",
    "Zatloukal Filip": "green",
    "Vlček Jiří": "cyan",
    "Štěpař Kamil": "brown",
    "Duč Jaroslav": "magenta",
    "Sako Dušan": "orange",
    "Diro Milan": "blue"
}

unique_people = df["Odpovědná osoba"].unique()
unique_people = [str(p) for p in unique_people]
people_data = {person: [] for person in unique_people}

def convert_value_to_float(val):
   
    s = str(val).replace("Kč","").replace("Eur","").replace(" ","").replace(",",".")
    try:
        return float(s)
    except:
        return None

def override_if_zero_or_empty(html_string, val) -> str:
   
    import re

    if val is None:
        
        return re.sub(r'(style="[^"]*color:)([^;"]*)(;?)', r'\1lightgray\3', html_string)
    
    val_s = str(val).strip()
    if val_s == "" or val_s == "N/A":
        return re.sub(r'(style="[^"]*color:)([^;"]*)(;?)', r'\1lightgray\3', html_string)

    float_val = convert_value_to_float(val_s)
    if float_val == 0:
        return re.sub(r'(style="[^"]*color:)([^;"]*)(;?)', r'\1lightgray\3', html_string)

    return html_string

def determine_border_color(row):
    
    last_active_year = None
    for y in reversed(years):
        val = row.get(f"Obrat_{y}", "N/A")
        if pd.notna(val) and val != "N/A" and val != '':
            last_active_year = y
            break

    if last_active_year is None:
        return "black"
    else:
        diff = current_year - last_active_year
        if diff == 3:
            return "red"
        elif diff == 2:
            return "orange"
        elif diff == 1:
            return "blue"
        else:
            return "green"

def fmt_row_bg(label, value, bg_color):
    
    fv = convert_value_to_float(value)
    if fv and fv > 0:
        return f'<div style="background-color:{bg_color}; color:black;"><b>{label} {value}</b></div>'
    else:
        return f'<div style="color:black;"><b>{label} {value}</b></div>'

def fmt_row_color_if_nonzero(label, value, color_when_nonzero):
    
    fv = convert_value_to_float(value)
    if fv and fv > 0:
        return f'<div style="color:{color_when_nonzero};"><b>{label} {value}</b></div>'
    else:
        return f'<div style="color:black;"><b>{label} {value}</b></div>'

def color_by_date(date_str):
    
    if not date_str or date_str == "N/A":
        return "black"
    try:
        dt = pd.to_datetime(date_str, format="%d.%m.%Y", errors="coerce")
        if pd.isnull(dt):
            dt = pd.to_datetime(date_str, format="%Y-%m-%d", errors="coerce")
        if pd.isnull(dt):
            return "black"
        diff = current_year - dt.year
        if diff == 0:
            return "green"
        elif diff == 1:
            return "blue"
        elif diff == 2:
            return "orange"
        else:
            return "red"
    except:
        return "black"

def is_zero_or_empty(value):
    
    if value is None:
        return True
    v_s = str(value).strip()
    if v_s == "" or v_s == "N/A":
        return True
    fl = convert_value_to_float(v_s)
    if fl == 0:
        return True
    return False

def is_positive_or_empty(value):
    
    if value is None:
        return True
    v_s = str(value).strip()
    if v_s == "" or v_s == "N/A":
        return True
    fl = convert_value_to_float(v_s)
    if fl is not None and fl > 0:
        return True
    return False

def fmt_cf_line(label, cf_str, obrat_str, uhrada_str):
     
    if is_zero_or_empty(uhrada_str) and is_zero_or_empty(obrat_str):
        return f'<div style="color:lightgray;"><b>{label} {cf_str}</b></div>'
    
    elif is_positive_or_empty(uhrada_str) and is_zero_or_empty(obrat_str):
        return f'<div style="color:green;"><b>{label} {cf_str}</b></div>'
    
    elif is_zero_or_empty(uhrada_str) and is_positive_or_empty(obrat_str):
        return f'<div style="color:red;"><b>{label} {cf_str}</b></div>'
    
    if cf_str.endswith("%"):
        try:
            val = int(cf_str.replace("%","").strip())
            if val < 100:
                return f'<div style="color:red;"><b>{label} {cf_str}</b></div>'
            else:
                return f'<div style="color:green;"><b>{label} {cf_str}</b></div>'
        except:
            return f'<div style="color:black;"><b>{label} {cf_str}</b></div>'
    else:
        return f'<div style="color:black;"><b>{label} {cf_str}</b></div>'

def build_tooltip_html(row):
        
    dt_fakt = row.get("Datum poslední faktury","N/A")
    col_fakt = color_by_date(dt_fakt)

    
    firma_html = f'<div style="color:{col_fakt};"><b>{row.get("firma","N/A")}</b></div>'
    adresa_html = f'<div style="color:{col_fakt};"><b>{row.get("adresa1","N/A")}</b></div>'
    
    osoba_html = f'<div style="color:black;"><b>Odpovědná osoba: {row.get("oz","N/A")}</b></div>'
    
    saldo_val = row.get("Celkové saldo", "N/A")
    fv_saldo = convert_value_to_float(saldo_val)
    if fv_saldo and fv_saldo > 0:
        saldo_html = f'<div style="color:red;"><b>Celkové saldo: {saldo_val}</b></div>'
    else:
        saldo_html = f'<div style="color:black;"><b>Celkové saldo: {saldo_val}</b></div>'
    saldo_html = override_if_zero_or_empty(saldo_html, saldo_val)

    sleva_val = row.get("Nejčastější sleva", "N/A")
    sleva_html = f'<div style="color:black;"><b>Nejčastější sleva: {sleva_val}</b></div>'
    sleva_html = override_if_zero_or_empty(sleva_html, sleva_val)

    rada_val = row.get("Poslední fakturační řada", "N/A")
    rada_html = f'<div style="color:black;"><b>Poslední fakturační řada: {rada_val}</b></div>'
    rada_html = override_if_zero_or_empty(rada_html, rada_val)

    proml_val = row.get("Promlčeno","N/A")
    promlceno_html = fmt_row_bg("Promlčeno:", proml_val, "red")
    promlceno_html = override_if_zero_or_empty(promlceno_html, proml_val)

    pred_val = row.get("Před promlčením","N/A")
    predproml_html = fmt_row_bg("Před promlčením:", pred_val, "orange")
    predproml_html = override_if_zero_or_empty(predproml_html, pred_val)

    nad_90_val = row.get("Nad 90 dní po splatnosti","N/A")
    nad_90_html = fmt_row_color_if_nonzero("Nad 90 dní po splatnosti:", nad_90_val, "red")
    nad_90_html = override_if_zero_or_empty(nad_90_html, nad_90_val)

    do_90_val = row.get("Do 90 dní po splatnosti","N/A")
    do_90_html = fmt_row_color_if_nonzero("Do 90 dní po splatnosti:", do_90_val, "orange")
    do_90_html = override_if_zero_or_empty(do_90_html, do_90_val)

    do_60_val = row.get("Do 60 dní po splatnosti","N/A")
    do_60_html = fmt_row_color_if_nonzero("Do 60 dní po splatnosti:", do_60_val, "blue")
    do_60_html = override_if_zero_or_empty(do_60_html, do_60_val)

    do_30_val = row.get("Do 30 dní po splatnosti","N/A")
    do_30_html = fmt_row_color_if_nonzero("Do 30 dní po splatnosti:", do_30_val, "green")
    do_30_html = override_if_zero_or_empty(do_30_html, do_30_val)

    ve_spl_val = row.get("Ve splatnosti","N/A")
    ve_spl_html = fmt_row_color_if_nonzero("Ve splatnosti:", ve_spl_val, "green")
    ve_spl_html = override_if_zero_or_empty(ve_spl_html, ve_spl_val)

    dt_fakt_html = f'<div style="color:{col_fakt};"><b>Datum poslední faktury: {dt_fakt}</b></div>'
    dt_fakt_html = override_if_zero_or_empty(dt_fakt_html, dt_fakt)

    vys_fakt_val = row.get("Výše poslední faktury","N/A")
    vys_fakt_html = f'<div style="color:{col_fakt};"><b>Výše poslední faktury: {vys_fakt_val}</b></div>'
    vys_fakt_html = override_if_zero_or_empty(vys_fakt_html, vys_fakt_val)

    dt_uhr = row.get("Datum poslední úhrady","N/A")
    col_uhr = color_by_date(dt_uhr)
    dt_uhr_html = f'<div style="color:{col_uhr};"><b>Datum poslední úhrady: {dt_uhr}</b></div>'
    dt_uhr_html = override_if_zero_or_empty(dt_uhr_html, dt_uhr)

    vys_uhr_val = row.get("Výše poslední úhrady","N/A")
    vys_uhr_html = f'<div style="color:{col_uhr};"><b>Výše poslední úhrady: {vys_uhr_val}</b></div>'
    vys_uhr_html = override_if_zero_or_empty(vys_uhr_html, vys_uhr_val)

    tooltip_html = f"""
<div style="text-align:left;">

{firma_html}
{adresa_html}
{osoba_html}
{saldo_html}
{sleva_html}
{rada_html}
<br>
{promlceno_html}
{predproml_html}
{nad_90_html}
{do_90_html}
{do_60_html}
{do_30_html}
{ve_spl_html}
<br>
{dt_fakt_html}
{vys_fakt_html}
{dt_uhr_html}
{vys_uhr_html}
<br>
"""

    for i, y in enumerate(years):
        obrat_val = row.get(f"Obrat_{y}", "N/A")
        pocet_val = row.get(f"Počet_{y}", "N/A")
        prumer_val = row.get(f"Průměr_{y}", "N/A")
        uhrady_val = row.get(f"Úhrady_{y}", "N/A")
        cf_raw     = row.get(f"C-F_{y}", "N/A")

        obrat_html = f'<div style="color:black;"><b>Obrat {y}:</b> {obrat_val}</div>'
        obrat_html = override_if_zero_or_empty(obrat_html, obrat_val)

        pocet_html = f'<div style="color:black;"><b>Počet {y}:</b> {pocet_val}</div>'
        pocet_html = override_if_zero_or_empty(pocet_html, pocet_val)

        prumer_html = f'<div style="color:black;"><b>Průměr {y}:</b> {prumer_val}</div>'
        prumer_html = override_if_zero_or_empty(prumer_html, prumer_val)

        uhrady_html = f'<div style="color:black;"><b>Úhrady {y}:</b> {uhrady_val}</div>'
        uhrady_html = override_if_zero_or_empty(uhrady_html, uhrady_val)

        cf_html = fmt_cf_line(f"C-F {y}:", cf_raw, obrat_val, uhrady_val)

        tooltip_html += obrat_html
        tooltip_html += pocet_html
        tooltip_html += prumer_html
        tooltip_html += uhrady_html
        tooltip_html += cf_html

        if y != list(years)[-1]:
            tooltip_html += "<br>"

    tooltip_html += "</div>"
    return tooltip_html

year_counts = {'no_active': 0}
for y in years:
    year_counts[y] = 0

for index, row in df.iterrows():
    lat_val = row["Latitude"]
    lon_val = row["Longitude"]
    if pd.notna(lat_val) and pd.notna(lon_val):
        lat = float(lat_val)
        lon = float(lon_val)
        person_str = row["Odpovědná osoba"]
        person_key = str(person_str) if pd.notna(person_str) else np.nan
        fill_color = person_color_map.get(person_key, "black")

        coord_key = (lat, lon)
        if coord_key in coordinate_counts:
            coordinate_counts[coord_key] += 1
        else:
            coordinate_counts[coord_key] = 0
        offset = 0.00008 * coordinate_counts[coord_key]
        adjusted_lon = lon + offset

        border_color = determine_border_color(row.to_dict())

        if border_color == "black":
            year_counts['no_active'] += 1
        elif border_color == "red":
            year_counts[current_year-3] += 1
        elif border_color == "orange":
            year_counts[current_year-2] += 1
        elif border_color == "blue":
            year_counts[current_year-1] += 1
        elif border_color == "green":
            year_counts[current_year] += 1

        dp_dict = {
            "lat": lat,
            "lon": adjusted_lon,
            "firma": to_serializable(row["Firma"]),
            "adresa1": to_serializable(row["Adresa 1"]),
            "oz": to_serializable(person_str),
            "Celkové saldo": to_serializable(row.get("Celkové saldo","N/A")),
            "Nejčastější sleva": to_serializable(row.get("Nejčastější sleva","N/A")),
            "Promlčeno": to_serializable(row.get("Promlčeno","N/A")),
            "Před promlčením": to_serializable(row.get("Před promlčením","N/A")),
            "Nad 90 dní po splatnosti": to_serializable(row.get("Nad 90 dní po splatnosti","N/A")),
            "Do 90 dní po splatnosti": to_serializable(row.get("Do 90 dní po splatnosti","N/A")),
            "Do 60 dní po splatnosti": to_serializable(row.get("Do 60 dní po splatnosti","N/A")),
            "Do 30 dní po splatnosti": to_serializable(row.get("Do 30 dní po splatnosti","N/A")),
            "Ve splatnosti": to_serializable(row.get("Ve splatnosti","N/A")),
            "Datum poslední faktury": to_serializable(row.get("Datum poslední faktury","N/A")),
            "Výše poslední faktury": to_serializable(row.get("Výše poslední faktury","N/A")),
            "Datum poslední úhrady": to_serializable(row.get("Datum poslední úhrady","N/A")),
            "Výše poslední úhrady": to_serializable(row.get("Výše poslední úhrady","N/A")),
            "Firma_Adresa": to_serializable(row["Firma_Adresa"]),
            "Poslední fakturační řada": to_serializable(row.get("Poslední fakturační řada","N/A"))
        }
        for y2 in years:
            dp_dict[f"Obrat_{y2}"] = to_serializable(row.get(f"Obrat_{y2}", "N/A"))
            dp_dict[f"Počet_{y2}"] = to_serializable(row.get(f"Počet_{y2}", "N/A"))
            dp_dict[f"Průměr_{y2}"] = to_serializable(row.get(f"Průměr_{y2}", "N/A"))
            dp_dict[f"Úhrady_{y2}"] = to_serializable(row.get(f"Úhrady_{y2}", "N/A"))
            dp_dict[f"C-F_{y2}"] = to_serializable(row.get(f"C-F_{y2}", "N/A"))

        people_data[str(person_str)].append(dp_dict)

        tooltip_html = build_tooltip_html(dp_dict)

        folium.CircleMarker(
            location=(lat, adjusted_lon),
            radius=5.5,
            color=border_color,
            fill=True,
            fill_opacity=0.7,
            fill_color=fill_color,
            tooltip=tooltip_html,
        ).add_to(map)

        pocet_pridanych_znacek += 1
    else:
        nezmapovane_radky.append(row)

nezmapovane_df = pd.DataFrame(nezmapovane_radky)

features = []
for person, data_points in people_data.items():
    for dp in data_points:
        properties = {
            "Firma": dp["firma"],
            "Adresa_1": dp["adresa1"],
            "Odpovedna_osoba": dp["oz"],
            "Celkove_saldo": dp["Celkové saldo"],
            "Nejcastejsi_sleva": dp["Nejčastější sleva"],
            "Promlceno": dp["Promlčeno"],
            "Pred_promlcenim": dp["Před promlčením"],
            "Nad_90": dp["Nad 90 dní po splatnosti"],
            "Do_90": dp["Do 90 dní po splatnosti"],
            "Do_60": dp["Do 60 dní po splatnosti"],
            "Do_30": dp["Do 30 dní po splatnosti"],
            "Ve_splatnosti": dp["Ve splatnosti"],
            "Datum_posledni_faktury": dp["Datum poslední faktury"],
            "Vyse_posledni_faktury": dp["Výše poslední faktury"],
            "Datum_posledni_uhrady": dp["Datum poslední úhrady"],
            "Vyse_posledni_uhrady": dp["Výše poslední úhrady"],
            "Firma_Adresa": dp["Firma_Adresa"],
            "Posledni_fakturacni_rada": dp["Poslední fakturační řada"]
        }
        for y2 in years:
            properties[f"Obrat_{y2}"] = dp[f"Obrat_{y2}"]
            properties[f"Počet_{y2}"] = dp[f"Počet_{y2}"]
            properties[f"Průměr_{y2}"] = dp[f"Průměr_{y2}"]
            properties[f"Úhrady_{y2}"] = dp[f"Úhrady_{y2}"]
            properties[f"C-F_{y2}"] = dp[f"C-F_{y2}"]

        ftr = {
            "type": "Feature",
            "properties": properties,
            "geometry": {
                "type": "Point",
                "coordinates": [dp["lon"], dp["lat"]]
            }
        }
        features.append(ftr)

geojson_layer = folium.GeoJson(
    {
        "type": "FeatureCollection",
        "features": features
    },
    name="Markery",
).add_to(map)

Search(
    layer=geojson_layer,
    geom_type="Point",
    search_label="Firma_Adresa",
    placeholder="Hledat",
    collapsed=False,
    position='topleft',
    marker=False
).add_to(map)

folium.LayerControl(position='bottomleft').add_to(map)

script = """
<script>
    document.addEventListener('DOMContentLoaded', function() {
        var layers = document.querySelectorAll('.leaflet-control-layers-overlays input');
        layers.forEach(function(layer) {
            if (layer.checked) {
                layer.click();
            }
        });
    });
</script>
"""

map.get_root().html.add_child(folium.Element(script))

map.get_root().html.add_child(folium.Element(f"""
<div style="position:absolute; bottom:35px; right:10px; z-index:9999; font-size:14px; font-weight:bold;">
<span style="color:black;">neaktiv.: {year_counts['no_active']}</span><br>
<span style="color:red;">{current_year-3}: {year_counts[current_year-3]}</span><br>
<span style="color:orange;">{current_year-2}: {year_counts[current_year-2]}</span><br>
<span style="color:blue;">{current_year-1}: {year_counts[current_year-1]}</span><br>
<span style="color:green;">aktivní: {year_counts[current_year]}</span>
</div>
"""))

map.get_root().html.add_child(folium.Element(f"""
<div style="position:absolute; bottom:10px; right:10px; z-index:9999; font-size:16px; color:red; font-weight:bold;">
{current_date}
</div>
"""))

map.get_root().html.add_child(folium.Element(f"""
<div style="position:absolute; bottom:10px; left:65px; z-index:9999; font-size:14px; color:red; font-weight:bold;">
<span style="color:red;">© Karel Mýl s.r.o.</span><br>
</div>
"""))

main_map_filename = f"výsledky/mapa_kompletní.html"

map.get_root().html.add_child(folium.Element("""
<style>
.leaflet-tooltip {
    font-size: 1.25em !important;
}
</style>
"""))

map.save(main_map_filename)

for person, data_points in people_data.items():
    person_map = folium.Map(location=[49.8175, 15.4730], zoom_start=7, zoom_control=False)
    person_map.get_root().html.add_child(folium.Element("""
    <style>
    .leaflet-control-attribution {
        display: none !important;
    }
    </style>
    """))

    person_features = []
    person_year_counts = {'no_active':0}
    for y2 in years:
        person_year_counts[y2] = 0

    person_key = str(person) if pd.notna(person) else np.nan
    pm_fill_color = person_color_map.get(person_key, "black")

    for dp in data_points:
        
        pseudo_row = {}
        for y2 in years:
            pseudo_row[f"Obrat_{y2}"] = dp[f"Obrat_{y2}"]
        border_color = determine_border_color(pseudo_row)

        if border_color == "black":
            person_year_counts['no_active'] += 1
        elif border_color == "red":
            person_year_counts[current_year-3] += 1
        elif border_color == "orange":
            person_year_counts[current_year-2] += 1
        elif border_color == "blue":
            person_year_counts[current_year-1] += 1
        elif border_color == "green":
            person_year_counts[current_year] += 1

        tooltip_html_person = build_tooltip_html(dp)

        folium.CircleMarker(
            location=(dp["lat"], dp["lon"]),
            radius=5.5,
            color=border_color,
            fill=True,
            fill_opacity=0.7,
            fill_color=pm_fill_color,
            tooltip=tooltip_html_person,
        ).add_to(person_map)

        person_properties = {
            "Firma": dp["firma"],
            "Adresa_1": dp["adresa1"],
            "Odpovedna_osoba": dp["oz"],
            "Celkove_saldo": dp["Celkové saldo"],
            "Nejcastejsi_sleva": dp["Nejčastější sleva"],
            "Promlceno": dp["Promlčeno"],
            "Pred_promlcenim": dp["Před promlčením"],
            "Nad_90": dp["Nad 90 dní po splatnosti"],
            "Do_90": dp["Do 90 dní po splatnosti"],
            "Do_60": dp["Do 60 dní po splatnosti"],
            "Do_30": dp["Do 30 dní po splatnosti"],
            "Ve_splatnosti": dp["Ve splatnosti"],
            "Datum_posledni_faktury": dp["Datum poslední faktury"],
            "Vyse_posledni_faktury": dp["Výše poslední faktury"],
            "Datum_posledni_uhrady": dp["Datum poslední úhrady"],
            "Vyse_posledni_uhrady": dp["Výše poslední úhrady"],
            "Firma_Adresa": dp["Firma_Adresa"],
            "Posledni_fakturacni_rada": dp["Poslední fakturační řada"]
        }
        for y3 in years:
            person_properties[f"Obrat_{y3}"] = dp[f"Obrat_{y3}"]
            person_properties[f"Počet_{y3}"] = dp[f"Počet_{y3}"]
            person_properties[f"Průměr_{y3}"] = dp[f"Průměr_{y3}"]
            person_properties[f"Úhrady_{y3}"] = dp[f"Úhrady_{y3}"]
            person_properties[f"C-F_{y3}"] = dp[f"C-F_{y3}"]

        p_ftr = {
            "type": "Feature",
            "properties": person_properties,
            "geometry": {
                "type": "Point",
                "coordinates": [dp["lon"], dp["lat"]]
            }
        }
        person_features.append(p_ftr)

    person_geojson = folium.GeoJson(
        {
            "type": "FeatureCollection",
            "features": person_features
        },
        name="Markery",
    ).add_to(person_map)

    Search(
        layer=person_geojson,
        geom_type="Point",
        search_label="Firma_Adresa",
        placeholder="Hledat",
        collapsed=False,
        position='topleft',
        marker=False
    ).add_to(person_map)

    folium.LayerControl(position='bottomleft').add_to(person_map)
    person_map.get_root().html.add_child(folium.Element(script))

    person_map.get_root().html.add_child(folium.Element("""
    <style>
    .leaflet-tooltip {
        font-size: 1.25em !important;
    }
    </style>
    """))

    person_map.get_root().html.add_child(folium.Element(f"""
    <div style="position:absolute; bottom:35px; right:10px; z-index:9999; font-size:14px; font-weight:bold;">
    <span style="color:black;">neaktiv.: {person_year_counts['no_active']}</span><br>
    <span style="color:red;">{current_year-3}: {person_year_counts[current_year-3]}</span><br>
    <span style="color:orange;">{current_year-2}: {person_year_counts[current_year-2]}</span><br>
    <span style="color:blue;">{current_year-1}: {person_year_counts[current_year-1]}</span><br>
    <span style="color:green;">aktivní: {person_year_counts[current_year]}</span>
    </div>
    """))

    person_map.get_root().html.add_child(folium.Element(f"""
    <div style="position:absolute; bottom:10px; right:10px; z-index:9999; font-size:16px; color:red; font-weight:bold;">
    {current_date}
    </div>
    """))

    person_map.get_root().html.add_child(folium.Element(f"""
    <div style="position:absolute; bottom:10px; left:65px; z-index:9999; font-size:14px; color:red; font-weight:bold;">
    <span style="color:red;">© Karel Mýl s.r.o.</span><br>
    </div>
    """))

    filename_person = str(person).replace(" ", "_")
    person_map.save(f"výsledky/mapa_{filename_person}.html")

count_adresa1 = (df["GeocodedFrom"] == "Adresa 1").sum()
count_adresa2 = (df["GeocodedFrom"] == "Adresa 2").sum()

print(f"Počet přidaných značek: {pocet_pridanych_znacek}")
print(f"Počet nezmapovaných řádků: {len(nezmapovane_df)}")
print(f"Počet značek z Adresa 1: {count_adresa1}")
print(f"Počet značek z Adresa 2: {count_adresa2}")